In [ ]:
import scvi
import scanpy as sc
import numpy as np

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    sc.pp.filter_genes(adata, min_counts=3)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
            adata,
            flavor="seurat",
            n_top_genes=2000,
            layer="counts",
            batch_key=batch,
            subset=True
    )

    scvi.model.SCVI.setup_anndata(
        adata,
        layer="counts",
        batch_key=batch
    )
    return adata
    
batch_key_dic = {'Immune_Human' : 'batch',
                 'Immune_human_mouse' : 'batch',
                 'Lung' : 'batch',
                 'Mouse_brain' : 'batch',
                 'Pancreas' : 'tech',
                 'Simulation1' : 'Batch',
                 'Simulation2' : 'Batch'}
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Immune_human_mouse' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Mouse_brain' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Simulation1' : 'Group',
                 'Simulation2' : 'Group'}    

In [ ]:
dataset = 'Immune_Human_removed_top_cells'
batch = 'batch'
cell_type = 'final_annotation'
adata = load_data(dataset,batch)

In [ ]:
from scvi.model import SCANVI
from scvi.model import SCVI
vae = SCVI(adata)
n_epochs_scVI = np.min([round((20000 / adata.n_obs) * 400), 400])  # 400
n_epochs_scANVI = int(np.min([10, np.max([2, round(n_epochs_scVI / 3.0)])]))
vae.train(max_epochs=n_epochs_scVI)
adata.obsm["X_emb"] = vae.get_latent_representation()

In [ ]:
 scanvae = scvi.model.SCANVI.from_scvi_model(
        scvi_model=vae,
        labels_key=cell_type,
        unlabeled_category="UnknownUnknown",  # pick anything definitely not in a dataset
    )
scanvae.train(max_epochs=n_epochs_scANVI)
adata.obsm["X_scANVI"] = scanvae.get_latent_representation()


In [ ]:
adata.write('scanvi-Immune_Human-removed_top_cells.h5ad')

In [ ]:
adata = sc.read_h5ad("scanvi-Immune_Human-removed_top_cells.h5ad")

In [ ]:
#its a file in this folder
import scIB

In [ ]:
adata

In [ ]:
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all = scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed='X_scANVI',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=True,  # backwards compatibility
        isolated_labels_f1_=True,
        isolated_labels_asw_=True,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=True,
)

In [ ]:
results

In [ ]:
import numpy as np
np.savetxt(dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt(dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt(dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi


latent_matrix = adata.obsm['X_scANVI'] 
labels = np.array(adata.obs[cell_type] )
K = np.size(np.unique(labels))
kmeans = KMeans(n_clusters=K, random_state=0).fit(latent_matrix)
y_pred = kmeans.labels_

print('Computing NMI ...')
NMI = nmi(labels.flatten(), y_pred.flatten())
print('NMI = {}'.format(NMI))

In [ ]:
sc.pp.neighbors(adata, use_rep='X_scANVI')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type, frameon=False)
sc.pl.umap(adata, color=batch, frameon=False)
sc.pl.umap(adata, color='cluster', frameon=False)

In [ ]:
import pandas as pd
adata.obs['temp'] =  y_pred
adata.obs['temp'] = adata.obs['temp'].astype(str)
sc.pl.umap(adata, color='temp', frameon=False)

In [ ]:
import numpy as np
cluster_key = 'cluster'
sc.tl.louvain(adata, resolution=0.01, key_added=cluster_key)
len(np.unique(adata.obs[cluster_key]))

In [ ]:
sc.pp.neighbors(adata, use_rep='X_emb')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type, frameon=False)
sc.pl.umap(adata, color=batch, frameon=False)
sc.pl.umap(adata, color='cluster', frameon=False)